In [0]:
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM, GRU
from keras.models import Sequential
from keras import optimizers
import numpy as np
import random
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from keras.layers.normalization import BatchNormalization
import tensorflow as tf

In [0]:
def split_train_val_id(n, val_percent=0.1):
    ids = [i for i in range(n)]
    random.shuffle(ids)
    piv = int(n*val_percent)
    if piv==0: 
        piv=1
    return {'train': ids[:-piv], 'val': ids[-piv:]}

In [0]:
datas = np.load('train_features.npy')
labels = np.load('train_labels.npy')

In [0]:
NUM_DATA_USE = 5000
INPUT_DIM_1D = datas.shape[1]*datas.shape[2]
OUTPUT_DIM = labels.shape[1]-1
val_percent = 0.2

train_val_id = split_train_val_id(NUM_DATA_USE, val_percent)
train_id = train_val_id['train']
val_id = train_val_id['val']

train_datas = np.zeros((len(train_id), INPUT_DIM_1D))
train_labels = np.zeros((len(train_id), OUTPUT_DIM))
for i, idx in enumerate(train_id):
    train_datas[i] = datas[idx].reshape(INPUT_DIM_1D)
    train_labels[i] = labels[idx][1:]
    
#reduce data information
train_datas = train_datas[:, :10000]


# #seven labels
day1_labels = np.zeros(len(train_id))
day2_labels = np.zeros(len(train_id))
day3_labels = np.zeros(len(train_id))
day4_labels = np.zeros(len(train_id))
day5_labels = np.zeros(len(train_id))
day6_labels = np.zeros(len(train_id))
day7_labels = np.zeros(len(train_id))

for i in range(len(train_id)):
    if train_labels[i, :4].sum()!=0:
        day1_labels[i] = 1
    if train_labels[i, 4:8].sum()!=0:
        day2_labels[i] = 1
    if train_labels[i, 8:12].sum()!=0:
        day3_labels[i] = 1
    if train_labels[i, 12:16].sum()!=0:
        day4_labels[i] = 1
    if train_labels[i, 16:20].sum()!=0:
        day5_labels[i] = 1
    if train_labels[i, 20:24].sum()!=0:
        day6_labels[i] = 1
    if train_labels[i, 24:28].sum()!=0:
        day7_labels[i] = 1

In [0]:
NUM_OF_VAL = len(val_id)
val_datas = np.zeros((NUM_OF_VAL, INPUT_DIM_1D))
val_labels = np.zeros((NUM_OF_VAL, OUTPUT_DIM))
for i, idx in enumerate(val_id):
    val_datas[i] = datas[idx].reshape(INPUT_DIM_1D)
    val_labels[i] = labels[idx][1:]
    
#reduce data information
val_datas = val_datas[:, :10000]

# #seven labels
val_day1_labels = np.zeros(NUM_OF_VAL)
val_day2_labels = np.zeros(NUM_OF_VAL)
val_day3_labels = np.zeros(NUM_OF_VAL)
val_day4_labels = np.zeros(NUM_OF_VAL)
val_day5_labels = np.zeros(NUM_OF_VAL)
val_day6_labels = np.zeros(NUM_OF_VAL)
val_day7_labels = np.zeros(NUM_OF_VAL)

for i in range(NUM_OF_VAL):
    if val_labels[i, :4].sum()!=0:
        val_day1_labels[i] = 1
    if val_labels[i, 4:8].sum()!=0:
        val_day2_labels[i] = 1
    if val_labels[i, 8:12].sum()!=0:
        val_day3_labels[i] = 1
    if val_labels[i, 12:16].sum()!=0:
        val_day4_labels[i] = 1
    if val_labels[i, 16:20].sum()!=0:
        val_day5_labels[i] = 1
    if val_labels[i, 20:24].sum()!=0:
        val_day6_labels[i] = 1
    if val_labels[i, 24:28].sum()!=0:
        val_day7_labels[i] = 1

In [0]:
def proba2onehot(pred, length, threshold):
    for i in range(length):
        if pred[i]>=threshold:
            pred[i] = 1
        else:
            pred[i] = 0
    return pred

In [34]:
model = Sequential()
model.add(GRU(units=200, input_shape=(1, train_datas.shape[1]), return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=100, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(64, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(32, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(7, activation = "relu"))

model.compile(loss='mse', optimizer='adam')
model.summary()

BATCH_SIZE = 32
NUM_EPOCH = 30


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_7 (GRU)                  (None, 1, 200)            6120600   
_________________________________________________________________
dropout_11 (Dropout)         (None, 1, 200)            0         
_________________________________________________________________
gru_8 (GRU)                  (None, 100)               90300     
_________________________________________________________________
dropout_12 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                6464      
_________________________________________________________________
batch_normalization_5 (Batch (None, 64)                256       
_________________________________________________________________
dropout_13 (Dropout)         (None, 64)                0         
__________

In [39]:
X_train = np.reshape(train_datas, (train_datas.shape[0], 1, train_datas.shape[1]))
X_val = np.reshape(val_datas, (val_datas.shape[0], 1, val_datas.shape[1]))
all_day_labels=np.concatenate((day1_labels.reshape(4000,1),day2_labels.reshape(4000,1),day3_labels.reshape(4000,1),day4_labels.reshape(4000,1),day5_labels.reshape(4000,1),day6_labels.reshape(4000,1),day7_labels.reshape(4000,1)),axis=1)
val_all_day_labels=np.concatenate((val_day1_labels.reshape(1000,1),val_day2_labels.reshape(1000,1),val_day3_labels.reshape(1000,1),val_day4_labels.reshape(1000,1),val_day5_labels.reshape(1000,1),val_day6_labels.reshape(1000,1),val_day7_labels.reshape(1000,1)),axis=1)
model.fit(X_train, all_day_labels, batch_size=BATCH_SIZE, epochs=NUM_EPOCH, validation_data = (X_val, val_all_day_labels))

Train on 4000 samples, validate on 1000 samples
Epoch 1/30
4000/4000 [==============================] - 6s 1ms/step - loss: 0.5136 - val_loss: 0.2221
Epoch 2/30
4000/4000 [==============================] - 3s 653us/step - loss: 0.2677 - val_loss: 0.1933
Epoch 3/30
4000/4000 [==============================] - 3s 654us/step - loss: 0.2247 - val_loss: 0.1698
Epoch 4/30
4000/4000 [==============================] - 3s 653us/step - loss: 0.2034 - val_loss: 0.1552
Epoch 5/30
4000/4000 [==============================] - 3s 646us/step - loss: 0.1929 - val_loss: 0.1438
Epoch 6/30
4000/4000 [==============================] - 3s 650us/step - loss: 0.1789 - val_loss: 0.1373
Epoch 7/30
4000/4000 [==============================] - 3s 651us/step - loss: 0.1696 - val_loss: 0.1303
Epoch 8/30
4000/4000 [==============================] - 3s 658us/step - loss: 0.1610 - val_loss: 0.1269
Epoch 9/30
4000/4000 [==============================] - 3s 652us/step - loss: 0.1522 - val_loss: 0.1286
Epoch 10/30
4000/4

In [47]:
THRESHOLD = 0.5
y_pred = model.predict(X_val)
y_pred = np.reshape(y_pred, (NUM_OF_VAL*7))
y_pred = proba2onehot(y_pred, NUM_OF_VAL*7, THRESHOLD)
print(classification_report(val_all_day_labels.reshape(7000), y_pred)) 

              precision    recall  f1-score   support

         0.0       0.86      0.94      0.90      5302
         1.0       0.75      0.53      0.62      1698

    accuracy                           0.84      7000
   macro avg       0.80      0.74      0.76      7000
weighted avg       0.83      0.84      0.83      7000



In [48]:
model = Sequential()
model.add(GRU(units=200, input_shape=(1, train_datas.shape[1]), return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=100, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(7, activation = "relu"))

model.compile(loss='mse', optimizer='adam')
model.summary()

BATCH_SIZE = 32
NUM_EPOCH = 20

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_9 (GRU)                  (None, 1, 200)            6120600   
_________________________________________________________________
dropout_15 (Dropout)         (None, 1, 200)            0         
_________________________________________________________________
gru_10 (GRU)                 (None, 100)               90300     
_________________________________________________________________
dropout_16 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 7)                 707       
Total params: 6,211,607
Trainable params: 6,211,607
Non-trainable params: 0
_________________________________________________________________


In [49]:
X_train = np.reshape(train_datas, (train_datas.shape[0], 1, train_datas.shape[1]))
X_val = np.reshape(val_datas, (val_datas.shape[0], 1, val_datas.shape[1]))
all_day_labels=np.concatenate((day1_labels.reshape(4000,1),day2_labels.reshape(4000,1),day3_labels.reshape(4000,1),day4_labels.reshape(4000,1),day5_labels.reshape(4000,1),day6_labels.reshape(4000,1),day7_labels.reshape(4000,1)),axis=1)
val_all_day_labels=np.concatenate((val_day1_labels.reshape(1000,1),val_day2_labels.reshape(1000,1),val_day3_labels.reshape(1000,1),val_day4_labels.reshape(1000,1),val_day5_labels.reshape(1000,1),val_day6_labels.reshape(1000,1),val_day7_labels.reshape(1000,1)),axis=1)
model.fit(X_train, all_day_labels, batch_size=BATCH_SIZE, epochs=NUM_EPOCH, validation_data = (X_val, val_all_day_labels))

Train on 4000 samples, validate on 1000 samples
Epoch 1/20
4000/4000 [==============================] - 5s 1ms/step - loss: 0.1910 - val_loss: 0.1340
Epoch 2/20
4000/4000 [==============================] - 2s 524us/step - loss: 0.1407 - val_loss: 0.1295
Epoch 3/20
4000/4000 [==============================] - 2s 531us/step - loss: 0.1340 - val_loss: 0.1231
Epoch 4/20
4000/4000 [==============================] - 2s 525us/step - loss: 0.1265 - val_loss: 0.1216
Epoch 5/20
4000/4000 [==============================] - 2s 540us/step - loss: 0.1245 - val_loss: 0.1223
Epoch 6/20
4000/4000 [==============================] - 2s 523us/step - loss: 0.1226 - val_loss: 0.1203
Epoch 7/20
4000/4000 [==============================] - 2s 524us/step - loss: 0.1202 - val_loss: 0.1179
Epoch 8/20
4000/4000 [==============================] - 2s 532us/step - loss: 0.1160 - val_loss: 0.1187
Epoch 9/20
4000/4000 [==============================] - 2s 531us/step - loss: 0.1146 - val_loss: 0.1136
Epoch 10/20
4000/4

In [50]:
THRESHOLD = 0.5
y_pred = model.predict(X_val)
y_pred = np.reshape(y_pred, (NUM_OF_VAL*7))
y_pred = proba2onehot(y_pred, NUM_OF_VAL*7, THRESHOLD)
print(classification_report(val_all_day_labels.reshape(7000), y_pred)) 

              precision    recall  f1-score   support

         0.0       0.88      0.92      0.90      5302
         1.0       0.71      0.59      0.65      1698

    accuracy                           0.84      7000
   macro avg       0.80      0.76      0.77      7000
weighted avg       0.84      0.84      0.84      7000

